In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# K-matrix

While {mod}`ampform` does not yet provide a generic way to produce an amplitude model with $\boldsymbol{K}$-matrix dynamics, the (experimental) {mod}`.kmatrix` module makes fairly simple to produce an expression for the $\boldsymbol{K}$-matrix and play around with it interactively. For more info on the $\boldsymbol{K}$-matrix, see this instructive presentation {cite}`meyerMatrixTutorial2008`, or the classic paper {cite}`chungPartialWaveAnalysis1995`.

In [ ]:
%matplotlib widget
import logging
import re
import warnings

import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import sympy as sp

import symplot

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

## Non-relativistic

::::{margin}
:::{seealso} {doc}`compwa-org:report/005`
:::
::::

This section shows how a $\boldsymbol{K}$-matrix for **one channel with two poles** compares to using a sum of two Breit-Wigner functions.

A non-relativistic $\boldsymbol{K}$-matrix for an arbitrary number of channels and an arbitrary number of resonances can be formulated with the {meth}`.NonRelativisticKMatrix.formulate` method:

In [ ]:
from ampform.dynamics.kmatrix import NonRelativisticKMatrix

n_resonances = sp.Symbol("n_R", integer=True, positive=True)
k_matrix_nr = NonRelativisticKMatrix.formulate(
    n_resonances=n_resonances, n_channels=1
)
k_matrix_nr[0, 0]

Notice how the $\boldsymbol{K}$-matrix reduces to a {func}`.relativistic_breit_wigner` when there is one channel and one resonance (but for a residue constant $\gamma$):

In [ ]:
k_matrix_1r = NonRelativisticKMatrix.formulate(n_resonances=1, n_channels=1)
k_matrix_1r[0, 0].doit().simplify()

Now let's investigate the effect of using a $\boldsymbol{K}$-matrix to describe **two resonances** in one channel and see how it compares with the sum of two Breit-Wigner functions. Two Breit-Wigner 'poles' with the same parameters would look like this:

In [ ]:
from ampform.dynamics import relativistic_breit_wigner

s, m1, m2, Gamma1, Gamma2, gamma1, gamma2 = sp.symbols(
    "s, m1, m2, Gamma1, Gamma2, gamma1, gamma2"
)
term1 = relativistic_breit_wigner(s, m1, Gamma1)
term2 = relativistic_breit_wigner(s, m2, Gamma2)
bw = gamma1 * term1 + gamma2 * term2
bw

while a $\boldsymbol{K}$-matrix parametrizes the two resonances as:

In [ ]:
k_matrix_2r = NonRelativisticKMatrix.formulate(n_resonances=2, n_channels=1)
k_matrix = k_matrix_2r[0, 0].doit()

In [ ]:
# reformulate terms
denominator, nominator = k_matrix.args
term1 = nominator.args[0] * denominator
term2 = nominator.args[1] * denominator
k_matrix = term1 + term2
k_matrix

To simplify things, we can set the residue constants $\gamma$ to one. Notice how the $\boldsymbol{K}$-matrix has introduced some coupling ('interference') between the two terms.

In [ ]:
def remove_residue_constants(expression):
    expression = symplot.substitute_indexed_symbols(expression)
    residue_constants = filter(
        lambda s: re.match(r"^\\?gamma", s.name),
        expression.free_symbols,
    )
    return expression.xreplace({gamma: 1 for gamma in residue_constants})


display(
    remove_residue_constants(bw),
    remove_residue_constants(k_matrix),
)

Now, just like in {doc}`/usage/interactive`, we use {mod}`symplot` to visualize the difference between the two expressions. The important thing is that the Argand plot on the right shows that **the $\boldsymbol{K}$-matrix conserves unitarity**.

Note that we have to call {func}`symplot.substitute_indexed_symbols` to turn the {class}`~sympy.tensor.indexed.Indexed` instances in this {obj}`~sympy.matrices.dense.Matrix` expression into {class}`~sympy.core.symbol.Symbol`s before calling this function. We also call {func}`symplot.rename_symbols` so that the residue $\gamma$'s get a name that does not have to be dummified by {func}`~sympy.utilities.lambdify.lambdify`.

In [ ]:
# Prepare expressions
m = sp.Symbol("m")
kmatrix = symplot.substitute_indexed_symbols(k_matrix)
rename_gamma = lambda s: re.sub(  # noqa: E731
    r"\\gamma_{([0-9]),0}", r"gamma\1", s
)
bw = symplot.rename_symbols(bw, rename_gamma)
kmatrix = symplot.rename_symbols(kmatrix, rename_gamma)
bw = bw.xreplace({s: m ** 2})
kmatrix = kmatrix.xreplace({s: m ** 2})

# Prepare sliders and domain
np_kmatrix, sliders = symplot.prepare_sliders(kmatrix, m)
np_bw = sp.lambdify((m, Gamma1, Gamma2, gamma1, gamma2, m1, m2), bw.doit())

m_min, m_max = 0, 3
domain_1d = np.linspace(m_min, m_max, 200)
domain_argand = np.linspace(m_min - 2, m_max + 2, 1_000)
sliders.set_ranges(
    m1=(0, 3, 100),
    m2=(0, 3, 100),
    Gamma1=(0, 2, 100),
    Gamma2=(0, 2, 100),
    gamma1=(0, 2),
    gamma2=(0, 2),
)
sliders.set_values(
    m1=1.1,
    m2=1.9,
    Gamma1=0.2,
    Gamma2=0.3,
    gamma1=1,
    gamma2=1,
)
if STATIC_WEB_PAGE:
    # Concatenate flipped domain for reverse animation
    domain = np.linspace(1.0, 2.7, 50)
    domain = np.concatenate((domain, np.flip(domain[1:])))
    sliders._sliders["m1"] = domain


def create_argand(func):
    def wrapped(**kwargs):
        values = func(domain_argand, **kwargs)
        argand = np.array([values.real, values.imag])
        return argand.T

    return wrapped


# Create figure
fig, axes = plt.subplots(
    ncols=2,
    figsize=1.2 * np.array((8, 3.8)),
    tight_layout=True,
)
ax_intensity, ax_argand = axes
m_label = "$m_{a+b}$"
ax_intensity.set_xlabel(m_label)
ax_intensity.set_ylabel("$|A|^2$")
ax_argand.set_xlabel("Re($A$)")
ax_argand.set_ylabel("Im($A$)")

# Plot intensity
controls = iplt.plot(
    domain_1d,
    lambda *args, **kwargs: np.abs(np_kmatrix(*args, **kwargs) ** 2),
    label="$K$-matrix",
    **sliders,
    ylim="auto",
    ax=ax_intensity,
)
iplt.plot(
    domain_1d,
    lambda *args, **kwargs: np.abs(np_bw(*args, **kwargs) ** 2),
    label="Breit-Wigner",
    controls=controls,
    ylim="auto",
    ax=ax_intensity,
)
plt.legend(loc="upper right")
iplt.axvline(controls["m1"], c="gray", linestyle="dotted")
iplt.axvline(controls["m2"], c="gray", linestyle="dotted")

# Argand plots
iplt.scatter(
    create_argand(np_kmatrix),
    label="$K$-matrix",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
)
iplt.scatter(
    create_argand(np_bw),
    label="Breit-Wigner",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
)
plt.legend(loc="upper right");

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import Image

    output_path = "k-matrix.gif"
    ax_intensity.set_ylim([0, 2])
    ax_argand.set_xlim([-1, +1])
    ax_argand.set_ylim([0, 2])
    controls.save_animation(output_path, fig, "m1", fps=20)
    with open(output_path, "rb") as f:
        display(Image(data=f.read(), format="png"))

## Relativistic

::::{margin}
:::{seealso} {doc}`compwa-org:report/009`
:::
::::

Relativistic $\boldsymbol{K}$-matrices for an arbitrary number of channels and an arbitrary number of resonances can be formulated with the {meth}`.RelativisticKMatrix.formulate` method:

In [ ]:
from ampform.dynamics.kmatrix import RelativisticKMatrix

n_resonances = sp.Symbol("n_R", integer=True, positive=True)
rel_k_matrix_nr = RelativisticKMatrix.formulate(
    n_resonances=n_resonances, n_channels=1
)
rel_k_matrix_nr[0, 0]

Again, as in {ref}`usage/dynamics/k-matrix:Non-relativistic`, the $\boldsymbol{K}$-matrix reduces to something of a {func}`.relativistic_breit_wigner`. This time, the width has been replaced by a {class}`.CoupledWidth` and some {class}`.PhaseSpaceFactor`s have been inserted that take case of the decay into two decay products:[^1]

[^1]: The two {class}`.PhaseSpaceFactor`s $\sqrt{\rho_0(s)}$ cancel out within phase space, where they are real.

In [ ]:
rel_k_matrix_1r = RelativisticKMatrix.formulate(n_resonances=1, n_channels=1)
symplot.partial_doit(rel_k_matrix_1r[0, 0], sp.Sum).simplify(doit=False)

Similarly, the $\boldsymbol{K}$-matrix with two resonances becomes (neglecting the $\sqrt{\rho_0(s)}$):

In [ ]:
rel_k_matrix_2r = RelativisticKMatrix.formulate(n_resonances=2, n_channels=1)
rel_k_matrix_2r = symplot.partial_doit(rel_k_matrix_2r[0, 0], sp.Sum)

In [ ]:
from ampform.dynamics import PhaseSpaceFactor

rel_k_matrix_2r = symplot.substitute_indexed_symbols(rel_k_matrix_2r)
s, m_a, m_b = sp.symbols("s, m_a0, m_b0")
rho = PhaseSpaceFactor(s, m_a, m_b)
rel_k_matrix_2r = rel_k_matrix_2r.xreplace(
    {
        sp.sqrt(rho): 1,
        sp.conjugate(sp.sqrt(rho)): 1,
    }
)
denominator, nominator = rel_k_matrix_2r.args
term1 = nominator.args[0] * denominator
term2 = nominator.args[1] * denominator
rel_k_matrix_2r = term1 + term2
rel_k_matrix_2r